In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
symbol = 'AAPL'
start_date = '2015-01-01'
end_date = '2023-12-31'
n_iter = 10  # Número de combinações a testar

In [3]:
df = yf.download(symbol, start=start_date, end=end_date)
df = df[['Close']]

# Normalização
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Função de janela
def create_dataset(data, time_step):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

C:\Users\Marcos Simoneli\AppData\Local\Temp\ipykernel_31348\1548301510.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [4]:
best_mape = float('inf')
best_config = None

for i in range(n_iter):
    print(f"\n🔁 Execução {i+1}/{n_iter}")

    # Hiperparâmetros aleatórios
    time_step = random.choice([30, 60, 90])
    lstm_units = random.choice([50, 64, 100, 128])
    dropout_rate = random.choice([0.1, 0.2, 0.3, 0.5])
    batch_size = random.choice([16, 32, 64])
    learning_rate = random.choice([0.001, 0.0005, 0.0001])
    epochs = 50

    # Dados
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size - time_step:]

    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Modelo
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=False, input_shape=(time_step, 1)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(X_train, y_train, 
              epochs=epochs, 
              batch_size=batch_size, 
              validation_split=0.1, 
              verbose=0, 
              callbacks=[es])

    # Avaliação
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))
    mape = mean_absolute_percentage_error(real_prices, predictions)

    print(f"MAPE: {mape:.4%}  | time_step={time_step}, lstm={lstm_units}, dropout={dropout_rate}, batch={batch_size}, lr={learning_rate}")

    if mape < best_mape:
        best_mape = mape
        best_config = {
            'MAPE': mape,
            'time_step': time_step,
            'lstm_units': lstm_units,
            'dropout': dropout_rate,
            'batch_size': batch_size,
            'learning_rate': learning_rate
        }

c:\Users\Marcos Simoneli\OneDrive\Documentos\Machine Learning\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



🔁 Execução 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
MAPE: 2.9282%  | time_step=30, lstm=50, dropout=0.3, batch=64, lr=0.001

🔁 Execução 2/10


c:\Users\Marcos Simoneli\OneDrive\Documentos\Machine Learning\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
MAPE: 2.3164%  | time_step=60, lstm=128, dropout=0.1, batch=32, lr=0.001

🔁 Execução 3/10


c:\Users\Marcos Simoneli\OneDrive\Documentos\Machine Learning\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
MAPE: 2.5873%  | time_step=60, lstm=100, dropout=0.2, batch=16, lr=0.0005

🔁 Execução 9/10


c:\Users\Marcos Simoneli\OneDrive\Documentos\Machine Learning\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
MAPE: 2.4811%  | time_step=90, lstm=100, dropout=0.5, batch=32, lr=0.001

🔁 Execução 10/10


c:\Users\Marcos Simoneli\OneDrive\Documentos\Machine Learning\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
MAPE: 2.3485%  | time_step=90, lstm=100, dropout=0.3, batch=16, lr=0.0005


In [5]:
print("\n✅ MELHOR CONFIGURAÇÃO:")
for k, v in best_config.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}" if 'MAPE' in k else f"{k}: {v}")
    else:
        print(f"{k}: {v}")


✅ MELHOR CONFIGURAÇÃO:
MAPE: 0.0163
time_step: 90
lstm_units: 50
dropout: 0.1
batch_size: 16
learning_rate: 0.001
